In [1]:
import copy
import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))

from utils.model_utils.evaluate import evaluate_model
from utils.model_utils.load_model import load_model
from utils.model_utils.model_config import ModelConfig
from utils.dataset_utils.load_dataset import load_data
from utils.decompose_utils.weight_remover import WeightRemoverBert
from utils.decompose_utils.concern_identification import ConcernIdentificationBert
from utils.decompose_utils.tangling_identification import TanglingIdentification
from transformers import AutoConfig
from utils.model_utils.save_module import save_module
from datetime import datetime
from utils.decompose_utils.concern_modularization import ConcernModularizationBert
from utils.decompose_utils.sampling import sampling_class
from utils.dataset_utils.load_dataset import (
    convert_dataset_labels_to_binary,
    extract_and_convert_dataloader,
)
import torch

In [2]:
model_name = "Salesforce/codet5-base-multi-sum"
task_type = "seq2seq"
architectures = "T5"
dataset_name = "code_search_net"
num_labels = 6

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
checkpoint = None
model_config = ModelConfig(
    model_name=model_name,
    task_type=task_type,
    dataset_name=dataset_name,
    checkpoint=checkpoint,
    device=device,
)

In [6]:
model, tokenizer, checkpoint = load_model(model_config)

Directory /home/Minwoo/LESN/Decompose/DecomposeTransformer/Models/Configs/seq2seq/Salesforce/codet5-base-multi-sum exists.
Loading the model.


In [7]:
from datasets import load_dataset

In [7]:
# dataset = load_dataset("code_search_net")
# print(dataset)

/home/Minwoo/.conda/envs/DecomposeTransformer/lib/python3.8/site-packages/datasets/load.py:1486: FutureWarning: The repository for code_search_net contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/code_search_net
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [8]:
train_dataloader, valid_dataloader, test_dataloader = load_data(
        model_config, batch_size=32
)

/home/Minwoo/.conda/envs/DecomposeTransformer/lib/python3.8/site-packages/datasets/load.py:1486: FutureWarning: The repository for code_search_net contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/code_search_net
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [49]:
import numpy as np

import re
from pathlib import Path


In [50]:
from transformers import RobertaTokenizer, T5ForConditionalGeneration

In [51]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [52]:
tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-base')
model = T5ForConditionalGeneration.from_pretrained("")

In [53]:
from utils.dataset_utils import load_dataset

In [ ]:
checkpoint_name = None
config = AutoConfig.from_pretrained(model_name, num_labels=num_labels)
model_config = ModelConfig(
    _model_name=model_name,
    _model_type=model_type,
    _data=data,
    _transformer_config=config,
    _checkpoint_name=checkpoint_name,
    _device=device,
)

In [55]:
model_name = "Salesforce/codet5-base-multi-sum"
model_type = "pretrained"
data = "OSDG"
num_labels = 16

In [ ]:
tokenizer = load_tokenizer(model_config)

In [18]:
import json

In [27]:
def process_jsonl_line(jsonl_line):
    """
    Process a single line of JSONL data.

    Args:
    jsonl_line (str): A single line from a JSONL file.

    Returns:
    dict: A dictionary with the processed data.
    """
    data = json.loads(jsonl_line)
    
    # Perform some operation on the data
    # Example: Extracting relevant information and transforming it
    processed_data = {
        "repository": data.get("repo"),
        "file_path": data.get("path"),
        "function_name": data.get("func_name"),
        "code_snippet": data.get("code"),
        "language": data.get("language"),
        "documentation": data.get("docstring"),
        "url": data.get("url")
    }
    
    return processed_data

In [28]:
def process_jsonl_file(file_path):
    """
    Process an entire JSONL file.

    Args:
    file_path (str): The path to the JSONL file.

    Returns:
    list: A list of dictionaries with the processed data from each line.
    """
    processed_data_list = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            processed_data = process_jsonl_line(line)
            processed_data_list.append(processed_data)
    
    return processed_data_list

In [32]:
file_path = '../Datasets/Codes/python/train.jsonl'

In [33]:
processed_data = process_jsonl_file(file_path)
for item in processed_data:
    print(item)
    break

{'repository': 'smdabdoub/phylotoast', 'file_path': 'phylotoast/util.py', 'function_name': 'split_phylogeny', 'code_snippet': 'def split_phylogeny(p, level="s"):\n    """\n    Return either the full or truncated version of a QIIME-formatted taxonomy string.\n\n    :type p: str\n    :param p: A QIIME-formatted taxonomy string: k__Foo; p__Bar; ...\n\n    :type level: str\n    :param level: The different level of identification are kingdom (k), phylum (p),\n                  class (c),order (o), family (f), genus (g) and species (s). If level is\n                  not provided, the default level of identification is species.\n\n    :rtype: str\n    :return: A QIIME-formatted taxonomy string up to the classification given\n            by param level.\n    """\n    level = level+"__"\n    result = p.split(level)\n    return result[0]+level+result[1].split(";")[0]', 'language': 'python', 'documentation': 'Return either the full or truncated version of a QIIME-formatted taxonomy string.\n\n  

In [34]:
a = {'repository': 'smdabdoub/phylotoast', 'file_path': 'phylotoast/util.py', 'function_name': 'split_phylogeny', 'code_snippet': 'def split_phylogeny(p, level="s"):\n    """\n    Return either the full or truncated version of a QIIME-formatted taxonomy string.\n\n    :type p: str\n    :param p: A QIIME-formatted taxonomy string: k__Foo; p__Bar; ...\n\n    :type level: str\n    :param level: The different level of identification are kingdom (k), phylum (p),\n                  class (c),order (o), family (f), genus (g) and species (s). If level is\n                  not provided, the default level of identification is species.\n\n    :rtype: str\n    :return: A QIIME-formatted taxonomy string up to the classification given\n            by param level.\n    """\n    level = level+"__"\n    result = p.split(level)\n    return result[0]+level+result[1].split(";")[0]', 'language': 'python', 'documentation': 'Return either the full or truncated version of a QIIME-formatted taxonomy string.\n\n    :type p: str\n    :param p: A QIIME-formatted taxonomy string: k__Foo; p__Bar; ...\n\n    :type level: str\n    :param level: The different level of identification are kingdom (k), phylum (p),\n                  class (c),order (o), family (f), genus (g) and species (s). If level is\n                  not provided, the default level of identification is species.\n\n    :rtype: str\n    :return: A QIIME-formatted taxonomy string up to the classification given\n            by param level.', 'url': 'https://github.com/smdabdoub/phylotoast/blob/0b74ef171e6a84761710548501dfac71285a58a3/phylotoast/util.py#L159-L177'}

In [43]:
print(a["code_snippet"])

def split_phylogeny(p, level="s"):
    """
    Return either the full or truncated version of a QIIME-formatted taxonomy string.

    :type p: str
    :param p: A QIIME-formatted taxonomy string: k__Foo; p__Bar; ...

    :type level: str
    :param level: The different level of identification are kingdom (k), phylum (p),
                  class (c),order (o), family (f), genus (g) and species (s). If level is
                  not provided, the default level of identification is species.

    :rtype: str
    :return: A QIIME-formatted taxonomy string up to the classification given
            by param level.
    """
    level = level+"__"
    result = p.split(level)
    return result[0]+level+result[1].split(";")[0]


In [ ]:
"repository": data.get("repo"),
"file_path": data.get("path"),
"function_name": data.get("func_name"),
"code_snippet": data.get("code"),
"language": data.get("language"),
"documentation": data.get("docstring"),
"url": data.get("url")

In [12]:
def test(model, tokenizer, text):
    model = model.to(device)
    input_ids = tokenizer(text, return_tensors="pt").input_ids.to(device)
    generated_ids = model.generate(input_ids, max_length=20)
    print(tokenizer.decode(generated_ids[0], skip_special_tokens=True))

In [44]:
text = '''def split_phylogeny(p, level="s"):
    """
    Return either the full or truncated version of a QIIME-formatted taxonomy string.

    :type p: str
    :param p: A QIIME-formatted taxonomy string: k__Foo; p__Bar; ...

    :type level: str
    :param level: The different level of identification are kingdom (k), phylum (p),
                  class (c),order (o), family (f), genus (g) and species (s). If level is
                  not provided, the default level of identification is species.

    :rtype: str
    :return: A QIIME-formatted taxonomy string up to the classification given
            by param level.
    """
    level = level+"__"
    result = p.split(level)
    return result[0]+level+result[1].split(";")[0]'''

In [45]:
test(model, tokenizer, text)

Splits a string into a full or truncated version of a taxonomy string by the given classification given


In [47]:
text = '''def split_phylogeny(p, level="s"):
    level = level+"__"
    result = p.split(level)
    return result[0]+level+result[1].split(";")[0]'''

In [48]:
test(model, tokenizer, text)

Split a PHYLOGENY string into its components.


In [10]:
import torch
from scipy.stats import norm


class WeightRemover:
    def __init__(self, model, device="cuda:0", p=0.8):
        self.model = model.to(device)
        self.device = device
        self.p = p
        self.results = {"layer": [], "input": [], "output": []}

    def hook(self, layer, input, output):
        self.results["layer"].append(layer)
        self.results["input"].append(input[0].to('cpu'))
        self.results["output"].append(output[0].to('cpu'))

    def register_hooks(self):
        handle_list = []
        for layer in self.model.modules():
            if isinstance(layer, torch.nn.Linear):
                handle = layer.register_forward_hook(self.hook)
                handle_list.append(handle)
        return handle_list

    def remove_hooks(self, handle_list):
        for handle in handle_list:
            handle.remove()

    def remove_weights(self, layer):
        current_weight = layer.weight.clone()
        if layer.bias is not None:
            current_bias = layer.bias.clone()
        else:
            current_bias = None

        mean = torch.mean(current_weight, dim=1, keepdim=True)
        std = torch.std(current_weight, dim=1, keepdim=True)
        z_scores = (current_weight - mean) / std

        lower_z, upper_z = norm.ppf(0.45), norm.ppf(0.55)
        mask = torch.logical_and(z_scores >= lower_z, z_scores < upper_z)

        current_weight[mask] = 0
        all_zeros = ~mask.any(dim=1)
        if current_bias is not None:
            current_bias[all_zeros] = 0
        self.set_parameters(layer, current_weight, current_bias)

    def set_parameters(self, layer, weight, bias):
        layer.weight.data = weight
        if bias is not None:
            layer.bias.data = bias

    def process(self, input_tensor, decoder_input_ids):
        self.results = {"layer": [], "input": [], "output": []}
        handle_list = self.register_hooks()
        output = self.model(input_ids=input_tensor.to(self.device), decoder_input_ids=decoder_input_ids.to(self.device))
        self.remove_hooks(handle_list)
        return output
        
    def apply_removal(self):
        total_original_weights = 0
        total_remaining_weights = 0

        for idx, layer in enumerate(self.results["layer"]):
            current_weight = layer.weight
            original_non_zero_weights = torch.sum(current_weight != 0).item()
            total_original_weights += original_non_zero_weights

            if torch.sum(current_weight != 0) > torch.numel(current_weight) * self.p:
                self.results["output"][idx] = self.results["output"][idx].to(self.device)
                self.remove_weights(layer)
                self.results["output"][idx] = self.results["output"][idx].to('cpu')

            remaining_non_zero_weights = torch.sum(layer.weight != 0).item()
            total_remaining_weights += remaining_non_zero_weights

            print(f"Layer {idx} - Original non-zero weights: {original_non_zero_weights}, Remaining non-zero weights: {remaining_non_zero_weights}, Reduction: {original_non_zero_weights - remaining_non_zero_weights}")

        print(f"Total original non-zero weights: {total_original_weights}")
        print(f"Total remaining non-zero weights: {total_remaining_weights}")
        print(f"Total reduction: {total_original_weights - total_remaining_weights}")


In [11]:
class ConcernIdentification:
    def __init__(self, ref_model, model, device='cuda:0', p=0.7):
        self.ref_model = ref_model.to(device)
        self.model = model.to(device)
        self.device = device
        self.p = p
        self.original_results = {"layer": [], "input": [], "output": []}
        self.current_results = {"layer": [], "input": [], "output": []}

    def original_hook(self, layer, input, output):
        self.original_results["layer"].append(layer)
        self.original_results["input"].append(input[0].to('cpu'))
        self.original_results["output"].append(output[0].to('cpu'))

    def current_hook(self, layer, input, output):
        self.current_results["layer"].append(layer)
        self.current_results["input"].append(input[0].to('cpu'))
        self.current_results["output"].append(output[0].to('cpu'))

    def register_hooks(self, model, hook):
        handle_list = []
        for layer in model.modules():
            if isinstance(layer, torch.nn.Linear):
                handle = layer.register_forward_hook(hook)
                handle_list.append(handle)
        return handle_list

    def remove_hooks(self, handle_list):
        for handle in handle_list:
            handle.remove()

    def prune(self, ref_model, model, original_output, output):
        current_weight = model.weight.clone()
        if model.bias is not None:
            current_bias = model.bias.clone()
        else:
            current_bias = None
        original_weight = ref_model.weight.clone()
        
        if ref_model.bias is not None:
            original_bias = ref_model.bias.clone()
        else:
            original_bias = None
        shape = current_weight.shape

        output_loss = output - original_output
        if len(output_loss.shape) > len(shape):
            output_loss = output_loss[:, 0, :]
            
        positive_loss_mask = (
            torch.all(output_loss > 0, dim=0).unsqueeze(1).expand(-1, shape[1])
        )

        original_weight_std = safe_std(original_weight, dim=1, keepdim=True)
        current_weight_std = safe_std(
            current_weight,
            epsilon=original_weight_std,
            unbiased=True,
            dim=1,
            keepdim=True,
        )

        padded_positive = torch.where(
            current_weight > 0, current_weight, torch.tensor(float("nan"))
        )
        padded_negative = torch.where(
            current_weight < 0, current_weight, torch.tensor(float("nan"))
        )
        positive_mean = torch.nanmean(padded_positive, dim=1, keepdim=True)
        negative_mean = torch.nanmean(padded_negative, dim=1, keepdim=True)

        positive_std = safe_std(
            current_weight,
            epsilon=current_weight_std,
            unbiased=True,
            dim=1,
            keepdim=True,
        )
        negative_std = safe_std(
            current_weight,
            epsilon=current_weight_std,
            unbiased=True,
            dim=1,
            keepdim=True,
        )

        positive_scores = (padded_positive - positive_mean) / positive_std
        negative_scores = (padded_negative - negative_mean) / negative_std

        positive_median = torch.nanmedian(padded_positive, dim=1, keepdim=True)
        negative_median = torch.nanmedian(padded_negative, dim=1, keepdim=True)
        lower_z, upper_z = norm.ppf(0.1), norm.ppf(0.3)

        positive_remove_mask = torch.where(
            positive_mean < positive_median.values,
            positive_scores <= lower_z,
            torch.logical_and(positive_scores >= lower_z, positive_scores < upper_z),
        )

        negative_remove_mask = torch.where(
            negative_mean < negative_median.values,
            torch.logical_and(negative_scores < -lower_z, negative_scores >= -upper_z),
            negative_scores >= -upper_z,
        )

        remove_mask = torch.where(
            ~positive_loss_mask, positive_remove_mask, negative_remove_mask
        )

        current_weight[remove_mask] = 0

        all_zeros = ~remove_mask.any(dim=1)
        if current_bias is not None:
            current_bias[all_zeros] = 0
        self.set_parameters(model, current_weight, current_bias)

    def set_parameters(self, layer, weight, bias):
        layer.weight.data = weight
        if bias is not None:
            layer.bias.data = bias

    def process(self, input_tensor, decoder_input_ids):
        self.original_results = {"layer": [], "input": [], "output": []}
        self.current_results = {"layer": [], "input": [], "output": []}

        handle_list = self.register_hooks(self.model, self.current_hook)
        self.model(input_ids=input_tensor.to(self.device), decoder_input_ids=decoder_input_ids.to(self.device))
        self.remove_hooks(handle_list)
        handle_list = self.register_hooks(self.ref_model, self.original_hook)
        self.ref_model(input_ids=input_tensor.to(self.device), decoder_input_ids=decoder_input_ids.to(self.device))
        self.remove_hooks(handle_list)

    def apply_prune(self):
        total_original_weights = 0
        total_remaining_weights = 0
        
        for idx, layer in enumerate(self.current_results["layer"]):
            current_weight = layer.weight
            original_non_zero_weights = torch.sum(current_weight != 0).item()
            total_original_weights += original_non_zero_weights
            
            if torch.sum(current_weight != 0) > torch.numel(current_weight) * self.p:
                self.original_results["output"][idx] = self.original_results["output"][idx].to(self.device)
                self.current_results["output"][idx] = self.current_results["output"][idx].to(self.device)
                self.prune(self.original_results["layer"][idx], layer, self.original_results["output"][idx],
                           self.current_results["output"][idx])
                self.original_results["output"][idx] = self.original_results["output"][idx].to('cpu')
                self.current_results["output"][idx] = self.current_results["output"][idx].to('cpu')

            remaining_non_zero_weights = torch.sum(layer.weight != 0).item()
            total_remaining_weights += remaining_non_zero_weights

            print(f"Layer {idx} - Original non-zero weights: {original_non_zero_weights}, Remaining non-zero weights: {remaining_non_zero_weights}, Reduction: {original_non_zero_weights - remaining_non_zero_weights}")
            
        
        print(f"Total original non-zero weights: {total_original_weights}")
        print(f"Total remaining non-zero weights: {total_remaining_weights}")
        print(f"Total reduction: {total_original_weights - total_remaining_weights}")                


def safe_std(tensor, epsilon=None, unbiased=False, dim=None, keepdim=True):
    if tensor.numel():
        return nanstd(tensor, dim=dim, unbiased=unbiased, keepdim=keepdim)
    else:
        return torch.tensor(epsilon, dtype=tensor.dtype)


def nanstd(tensor, unbiased=False, dim=None, keepdim=True):
    mask = torch.isnan(tensor)
    n_obs = mask.logical_not().sum(dim=dim, keepdim=keepdim)
    mean = torch.nanmean(tensor, dim=dim, keepdim=keepdim)

    centered = tensor - mean
    centered = centered.masked_fill(mask, 0)
    sum_sq = torch.sum(centered ** 2, dim=dim, keepdim=keepdim)

    unbiased_factor = torch.where(n_obs > 1, n_obs - 1, n_obs)
    var = sum_sq / unbiased_factor

    std = torch.sqrt(var)
    if not keepdim:
        std = std.squeeze(dim)
    return std

In [12]:
weight_remover = WeightRemover(model, device, 0.8)

In [13]:
batch_size = 4
sequence_length = 10

for idx in range(5):
    print(f"-------------{idx} of the ids-------------")
    random_input_ids = torch.tensor(np.random.randint(0, tokenizer.vocab_size, (batch_size, sequence_length)), dtype=torch.long)
    random_decoder_input_ids = torch.tensor(np.random.randint(0, tokenizer.vocab_size, (batch_size, sequence_length)), dtype=torch.long)
    with torch.no_grad():
        y_ = weight_remover.process(random_input_ids, random_decoder_input_ids)
    weight_remover.apply_removal()

-------------0 of the ids-------------
Layer 0 - Original non-zero weights: 589824, Remaining non-zero weights: 529869, Reduction: 59955
Layer 1 - Original non-zero weights: 589824, Remaining non-zero weights: 530119, Reduction: 59705
Layer 2 - Original non-zero weights: 589824, Remaining non-zero weights: 530293, Reduction: 59531
Layer 3 - Original non-zero weights: 589824, Remaining non-zero weights: 529914, Reduction: 59910
Layer 4 - Original non-zero weights: 2359296, Remaining non-zero weights: 2124269, Reduction: 235027
Layer 5 - Original non-zero weights: 2359296, Remaining non-zero weights: 2120295, Reduction: 239001
Layer 6 - Original non-zero weights: 589824, Remaining non-zero weights: 530228, Reduction: 59596
Layer 7 - Original non-zero weights: 589824, Remaining non-zero weights: 530697, Reduction: 59127
Layer 8 - Original non-zero weights: 589824, Remaining non-zero weights: 530473, Reduction: 59351
Layer 9 - Original non-zero weights: 589824, Remaining non-zero weights: 

In [14]:
test(ref_model, text)

2024-05-28 19:48:18.582816: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-28 19:48:18.620355: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-28 19:48:19.211259: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Converts a SVG string to a QImage.


In [15]:
test(model, text)

Converts a SVG string to a QImage.


In [16]:
import matplotlib.pyplot as plt

def parse_weight_data(data):
    epochs = []
    original_weights = []
    remaining_weights = []
    reductions = []

    pattern = re.compile(r'Total original non-zero weights: (\d+)\s+Total remaining non-zero weights: (\d+)\s+Total reduction: (\d+)')
    matches = pattern.findall(data)

    for i, match in enumerate(matches):
        epoch = i + 1
        original = int(match[0])
        remaining = int(match[1])
        reduction = int(match[2])

        epochs.append(epoch)
        original_weights.append(original)
        remaining_weights.append(remaining)
        reductions.append(reduction)

    return epochs, original_weights, remaining_weights, reductions

def plot_weight_changes(data):
    epochs, original_weights, remaining_weights, reductions = parse_weight_data(data)

    plt.figure(figsize=(10, 6))
    plt.plot(epochs, original_weights, label='Original Weights', marker='o')
    plt.plot(epochs, remaining_weights, label='Remaining Weights', marker='o')
    plt.plot(epochs, reductions, label='Reduction', marker='o')
    plt.xlabel('Epoch')
    plt.ylabel('Number of Weights')
    plt.title('Weight Changes per Epoch')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()


In [19]:
train_file = '../Datasets/Codes/python/train.jsonl'
valid_file = '../Datasets/Codes/python/valid.jsonl'
test_file = '../Datasets/Codes/python/test.jsonl'

In [20]:
import json

def load_code_snippets(file_path):
    code_snippets = []
    tokens = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            data = json.loads(line.strip())
            code_snippets.append(data['code_tokens'])
            tokens.append(data['docstring_tokens'])
    return code_snippets, tokens

In [21]:
train_snippets, train_tokens = load_code_snippets(train_file)
valid_snippets, valid_tokens = load_code_snippets(valid_file)
test_snippets, test_tokens = load_code_snippets(test_file)

In [22]:
len(train_snippets)

251820

In [23]:
train_snippets[0]

['def',
 'split_phylogeny',
 '(',
 'p',
 ',',
 'level',
 '=',
 '"s"',
 ')',
 ':',
 'level',
 '=',
 'level',
 '+',
 '"__"',
 'result',
 '=',
 'p',
 '.',
 'split',
 '(',
 'level',
 ')',
 'return',
 'result',
 '[',
 '0',
 ']',
 '+',
 'level',
 '+',
 'result',
 '[',
 '1',
 ']',
 '.',
 'split',
 '(',
 '";"',
 ')',
 '[',
 '0',
 ']']

In [24]:
ci = ConcernIdentification(ref_model, model, device='cuda:0', p=0.7)

In [25]:
for idx, (text, tokens) in enumerate(zip(train_snippets, train_tokens)):
    print(idx)
    input_ids = tokenizer(text, return_tensors="pt").input_ids.to(device)
    print(f"-------------{idx} of the ids-------------")
    random_input_ids = input_ids
    random_decoder_input_ids = generated_ids
    with torch.no_grad():
        y_ = ci.process(text, tokens)
    ci.apply_prune()
    if idx > 5:
        break

0


ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`input_ids` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

In [ ]:
test(model, text)

In [ ]:
test(ref_model, text)